# ライトカーブを描く

- ds9のregionで星を選択


In [ ]:
#module import
 
import numpy as np
import matplotlib.pyplot as plt
from SI_function import ImageReduction as IR
from SI_function import LightCurveFunction as LCF


In [ ]:
"""================================="""
input_list = 'PATH/out/light_dfs.list'   #skyまで引いたデータ　時系列で並べる
reg_file = 'PATH/select.reg'  #ds9で作ったcircle region greenがtarget星, それ以外の色がreference星
displacement_file = 'PATH/displacement.txt'   #ここに計算したdipslacementを格納  あるいはこれを読み込んで使用。
r_pix = False   #pix 重心を決定する際に使う半径 これをfalseにした場合は regionの半径を採用する。
half_aperture_fwhm = 10  #FWHMを求める際に使う半径　基本的に同程度の明るさの星を扱うのでこれは固定で良い　返ってきた結果がおかしい時は調整する ただしあまりここは動作が良くないぽい
half_aperture = False  #pix 開口測光の"半径" これをfalseにした場合は、3*FWHMで測光する
local_bkg = True #測光する際に周りを使うかどうか

In [ ]:
#開く
#PATHの設定
PATH, output_name = IR.set_PATH_name(input_list, output_file='dummy', type = 'light')
#画像オープン
images, hdrs = IR.open_fits_multi(input_list, PATH)
#skyを引いた際の誤差を修正
images = LCF.sky_offset(images, hdrs)

In [ ]:
#画像のずれを計算して変移を取得
displacements = LCF.calc_read_displacements(images, displacement_file)

In [ ]:
#region ファイルを開いて測光するポイントを取得
circles = LCF.open_reg_circle(reg_file)
#regionファイルでターゲット星を示すインデックスを返す
target = LCF.get_target(circles)

In [ ]:
photometries = LCF.do_aperture_photometry(images, displacements, circles, target, r_pix, half_aperture, half_aperture_fwhm, local_bkg = local_bkg)

In [ ]:
#ラベル用
dates = LCF.get_hdr_date(hdrs)
label = []
date_seconds = []
for i in range(len(dates)):
    label.append(dates[i].show_date())
    date_seconds.append(dates[i].return_seconds())
    

In [ ]:
fig=plt.figure(figsize=(10,3),dpi = 300)
fig.patch.set_facecolor('white')
plt.plot(date_seconds, np.array(photometries)[:,1]/np.array(photometries)[:,2])
#plt.plot(date_seconds, np.array(flux_target))
plt.xticks(date_seconds[0:len(date_seconds):10], label[0:len(date_seconds):10],rotation = -90)